In [1]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
time = "2024-12-31_13:33"
time = "2025-01-02_08:33"
time = "2025-01-03_06:00"
time = "2025-01-04_09:44"


raw_df = pd.read_csv(f"/projects/genomic-ml/da2343/ml_project_2/unsupervised/kmeans/results/{time}_results.csv")
raw_df

,window,train_actual_return,train_num_trades,train_direction,test_actual_return,test_num_trades,test_direction,train_average_return,train_sharpe_ratio,train_profit_factor,...,total_windows,reverse_test,num_clusters,clustering_algorithm,train_period,test_period,random_seed,instrument,num_perceptually_important_points,price_history_length
0,301,0.139376,305,short,-0.000546,34,long,0.290947,2.015289,1000.0,...,65,True,5,birch,14,1,42,GBP_CHF_M15,4,24
1,302,0.447243,741,short,-0.026922,71,long,0.290947,2.015289,1000.0,...,65,True,5,birch,14,1,42,GBP_CHF_M15,4,24
2,303,0.313068,591,short,0.023545,27,long,0.290947,2.015289,1000.0,...,65,True,5,birch,14,1,42,GBP_CHF_M15,4,24
3,304,0.320753,764,short,-0.003050,50,long,0.290947,2.015289,1000.0,...,65,True,5,birch,14,1,42,GBP_CHF_M15,4,24
4,306,0.080427,199,short,-0.000741,16,long,0.290947,2.015289,1000.0,...,65,True,5,birch,14,1,42,GBP_CHF_M15,4,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,397,0.187483,386,short,-0.034497,46,long,0.158404,1.143604,1000.0,...,94,True,5,birch,8,1,42,AUD_NZD_M15,4,24
1547,398,0.277431,462,short,0.009117,23,long,0.158404,1.143604,1000.0,...,94,True,5,birch,8,1,42,AUD_NZD_M15,4,24
1548,399,0.014044,56,long,0.000000,0,short,0.158404,1.143604,1000.0,...,94,True,5,birch,8,1,42,AUD_NZD_M15,4,24
1549,400,0.192650,357,short,-0.012541,22,long,0.158404,1.143604,1000.0,...,94,True,5,birch,8,1,42,AUD_NZD_M15,4,24


In [3]:
def analyze_performance(raw_df):
    # Group by relevant parameters and calculate averages
    group_cols = ['instrument', 'num_clusters', 'clustering_algorithm', 'train_period', 
                 'reverse_test', 'random_seed']
    
    # Calculate mean performance metrics for each parameter combination
    results_df = raw_df.groupby(group_cols).agg({
        'test_profit_factor': 'mean',
        'test_sharpe_ratio': 'mean',
        'test_win_ratio': 'mean',
        'test_num_trades': 'mean'
    }).reset_index()
    
    # Calculate combined score
    results_df['combined_score'] = (
        results_df['test_profit_factor'] * 0.4 +
        results_df['test_sharpe_ratio'] * 0.4 +
        (results_df['test_num_trades'] / results_df['test_num_trades'].max()) * 0.2
    )
    
    # Create display DataFrame with proper formatting
    display_df = pd.DataFrame({
        'Instrument': results_df['instrument'],
        'Clusters': results_df['num_clusters'],
        'Algorithm': results_df['clustering_algorithm'],
        'Train Period': results_df['train_period'].astype(str) + 'w',
        'Rev.Test': results_df['reverse_test'],
        'Seed': results_df['random_seed'],
        'PF': results_df['test_profit_factor'].round(3),
        'Sharpe': results_df['test_sharpe_ratio'].round(3),
        'Win%': (results_df['test_win_ratio'] * 100).round(1),
        'Trades': results_df['test_num_trades'].round(1),
        'Score': results_df['combined_score'].round(3)
    })
    
    # Sort by instrument name
    display_df = display_df.sort_values('Instrument')
    
    # Apply styling with color gradients
    styled_df = display_df.style\
        .format({
            'PF': '{:.3f}',
            'Sharpe': '{:.3f}',
            'Win%': '{:.1f}%',
            'Trades': '{:.1f}',
            'Score': '{:.3f}'
        })\
        .background_gradient(subset=['Score'], cmap='YlOrRd')\
        .background_gradient(subset=['PF'], cmap='YlOrRd')\
        .background_gradient(subset=['Sharpe'], cmap='YlOrRd')\
        .background_gradient(subset=['Win%'], cmap='YlOrRd')\
        .background_gradient(subset=['Trades'], cmap='YlOrRd')\
        .set_properties(**{
            'text-align': 'right',
            'font-family': 'monospace',
            'padding': '5px'
        })\
        .hide(axis="index")
    
    return results_df, styled_df

# Example usage:
results_df, styled_df = analyze_performance(raw_df)
styled_df  # Display the styled table

Instrument,Clusters,Algorithm,Train Period,Rev.Test,Seed,PF,Sharpe,Win%,Trades,Score
AUD_CAD_M15,6,birch,12w,False,42,1.380,0.113,56.5%,26.8,0.748
AUD_JPY_M15,5,birch,9w,True,42,1.303,0.090,60.2%,30.3,0.727
AUD_NZD_M15,5,birch,8w,True,42,1.307,0.074,55.9%,27.0,0.705
AUD_USD_M15,6,birch,9w,True,42,1.656,0.178,48.9%,24.8,0.873
CAD_JPY_M15,5,birch,4w,False,42,1.252,0.087,50.5%,30.8,0.709
EUR_CAD_M15,5,birch,7w,True,42,1.108,0.035,57.7%,14.0,0.536
EUR_CHF_M15,6,birch,4w,False,42,1.346,0.104,46.5%,27.1,0.732
EUR_GBP_M15,7,birch,9w,False,42,0.512,-0.249,45.7%,21.1,0.224
EUR_JPY_M15,9,birch,6w,False,42,0.624,-0.171,43.6%,16.0,0.271
EUR_USD_M15,5,birch,11w,False,42,0.756,-0.093,40.3%,14.6,0.348


In [4]:
print(results_df)

     instrument  num_clusters clustering_algorithm  train_period  \
0   AUD_CAD_M15             6                birch            12   
1   AUD_JPY_M15             5                birch             9   
2   AUD_NZD_M15             5                birch             8   
3   AUD_USD_M15             6                birch             9   
4   CAD_JPY_M15             5                birch             4   
5   EUR_CAD_M15             5                birch             7   
6   EUR_CHF_M15             6                birch             4   
7   EUR_GBP_M15             7                birch             9   
8   EUR_JPY_M15             9                birch             6   
9   EUR_USD_M15             5                birch            11   
10  GBP_CHF_M15             5                birch            14   
11  GBP_JPY_M15             5                birch             9   
12  GBP_USD_M15             9                birch             5   
13  NZD_USD_M15             8                bir